#Mouafak Alsaid Hasan

##ENGR421 Homework-02

###Linear Discrimination by Regression

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Definitions

In [70]:
def sigmond(W, w0, x):
  return 1/(1 + np.exp(-(W.T * x + w0))) 

eta = 0.01
epsilon = 1e-3


##Importing the Data sets

In [67]:
#impoting the image data from hw02_data_set_images csv file.
image_data_set = np.genfromtxt("hw02_data_set_images.csv", delimiter = ",")

#impoting labels of the image data from hw02_data_set_labels csv file.
labels_data_set = np.genfromtxt("hw02_data_set_labels.csv",dtype= 'str', delimiter = ",").astype(str)


# creating arrays to seperate test and traing data sets
training_set_data, test_set_data  = [], []
training_set_labels, test_set_labels = [], []


# fill in the arrays above.
for i in range(len(image_data_set)):
  if i <25 or (i >= 39 and i < 64) or (i >= 78 and i< 103) or (i >= 117 and i< 142) or (i >= 156 and i < 181):
    training_set_data.append(image_data_set[i])
    training_set_labels.append(labels_data_set[i])
  else:
    test_set_data.append(image_data_set[i])
    test_set_labels.append(labels_data_set[i])






##Discrimination by Regression Algorithm